In [65]:
from control import *
from sympy import *
from sympy.physics.mechanics import dynamicsymbols, init_vprinting
import numpy as np
from scipy.interpolate import interp1d
init_vprinting()

In [125]:
t_data = [0,10,20,30,40,50,60,70,80,90]
rho_data = [999.9,999.7,998.2,995.7,992.2,988.1,983.2,977.8,971.8,965.3]
cp_data = [4.212,4.191,4.183,4.174,4.174,4.174,4.179,4.187,4.195,4.208]
v_data = np.array([1.789,1.306,1.006,.805,.659,.556,.478,.415,.365,.326])*1e6
fr = interp1d(t_data,rho_data,kind='cubic')
fc = interp1d(t_data,cp_data,kind='cubic')
fv = interp1d(t_data,v_data,kind='cubic')
Avg = lambda fi,t1,t2: fi*fr((t1+t2)/2)*fc((t1+t2)/2)*abs(t2-t1)/3.6e6
def Dln(n, ti1, ti2, ti3, ti4):
    p = np.hstack([np.arange(0,  5), np.arange(10,15)])
    for i in p:
        if n is i:
            din  = ti1 - ti3
            dout = ti2 - ti4
            return (din-dout)/np.log(din/dout)
        else:
            din  = ti1 - ti4
            dout = ti2 - ti3
            return (din-dout)/np.log(din/dout)
# Dln = lambda th1, th2, tc1, tc2: np.round((abs(th1-tc1)-abs(th2-tc2))/np.log(abs(th1-tc1)/abs(th2-tc2)), 2)
K = lambda qh, a, dln: qh/a/dln
avg, dln, k = np.vectorize(Avg), np.vectorize(Dln), np.vectorize(K)

In [244]:
a = np.array([[490,860,39.5,37.4,30.6,32],
[500,860,43.7,41.1,34.7,36.2],
[510,860,47,43.4,32.8,35.1],
[520,860,55.1,49.4,33.7,37.5],
[530,860,55.2,49.7,34.6,38.2],
[560,710,54.9,49.7,34.7,39],
[570,710,54.9,49.8,35.1,39.4],
[580,710,54.9,50,35.4,39.7],
[590,710,54.9,50.1,35.7,39.9],
[600,710,55,50.2,35.8,40],
              
[390,770,54.9,48,35.1,38.4],
[380,770,55.1,48.2,35.1,38.6],
[370,770,54.7,48,35.6,39],
[360,770,54.7,48.1,35.7,39.1],
[350,770,54.8,48,35.9,39.2],
              
[390,770,54.4,48.3,35.4,38.5],
[380,770,54.6,48.4,36.2,39.7],
[370,770,54.5,48.4,36.3,39.6],
[360,770,54.4,47.5,35,38.2],
[350,770,54.4,47.6,35.2,38.4]])
hot, cold, ti = [0,2,3], [1,4,5], [2,3,4,5]
A = np.hstack([np.ones(10)*(9+10)*4500, np.ones(10)*(11+12)*3500])*np.pi/2e6
qh = avg(*a[:,hot].T)
qc = avg(*a[:,cold].T)
n = (qh/qc)*100
dln = Dln(np.arange(len(a)),*a[:,ti].T)
k = K(qh,A,dln)*1000
np.round([qh[-10:],qc[-10:],n[-10:],dln[-10:]],2).T,np.round(k[-10:],2)

(array([[  3.08,   2.93, 105.29,  14.63],
        [  3.  ,   3.1 ,  96.72,  14.73],
        [  2.84,   3.01,  94.16,  13.99],
        [  2.72,   3.01,  90.25,  13.94],
        [  2.73,   2.93,  93.14,  13.78],
        [  2.72,   2.75,  99.09,  14.35],
        [  2.7 ,   3.1 ,  86.94,  13.51],
        [  2.58,   2.93,  88.34,  13.45],
        [  2.84,   2.84, 100.24,  14.27],
        [  2.73,   2.84,  96.05,  14.12]]),
 array([1665.94, 1611.15, 1605.11, 1543.5 , 1564.38, 1501.43, 1579.61,
        1519.29, 1576.48, 1526.12]))

(array([[  3.21,   3.01, 106.63,  14.62],
        [  2.92,   3.19,  91.32,  14.49],
        [  2.84,   3.01,  94.16,  13.99],
        [  2.76,   3.01,  91.62,  13.88],
        [  2.73,   2.93,  93.14,  13.78],
        [  2.9 ,   3.1 ,  93.55,  13.54],
        [  2.78,   3.1 ,  89.75,  13.4 ],
        [  2.58,   2.93,  88.34,  13.45],
        [  2.84,   2.84, 100.24,  14.27],
        [  2.73,   2.84,  96.05,  14.12]]),
 array([1739.14, 1590.47, 1605.11, 1573.03, 1564.38, 1694.75, 1643.69,
        1519.29, 1576.48, 1526.12]))

(array([[  3.04,   3.1 ,  97.83,  14.49],
        [  2.92,   3.19,  91.32,  14.49],
        [  2.84,   3.01,  94.16,  13.99],
        [  2.76,   3.01,  91.62,  13.88],
        [  2.73,   2.93,  93.14,  13.78],
        [  2.9 ,   3.1 ,  93.55,  13.54],
        [  2.78,   3.1 ,  89.75,  13.4 ],
        [  2.58,   2.93,  88.34,  13.45],
        [  2.84,   2.84, 100.24,  14.27],
        [  2.73,   2.84,  96.05,  14.12]]),
 array([1657.43, 1590.47, 1605.11, 1573.03, 1564.38, 1694.75, 1643.69,
        1519.29, 1576.48, 1526.12]))

In [110]:
A

array([134303.08594096, 134303.08594096, 134303.08594096, 134303.08594096,
       134303.08594096, 134303.08594096, 134303.08594096, 134303.08594096,
       134303.08594096, 134303.08594096, 126449.10430699, 126449.10430699,
       126449.10430699, 126449.10430699, 126449.10430699, 126449.10430699,
       126449.10430699, 126449.10430699, 126449.10430699, 126449.10430699])

In [58]:
15.932 - (7.8-4.9)/np.log(7.8/4.9)